## pdf to text script
*goal: use OCR edited pdf files and extract individual sentences & paragraphs to a table*

This notebook is used to transform readable pdf documents into dataframes where each row either represents a sentence or a paragraph

In [1]:
# # function to remove unicode words like: https://github.com/traviscoan/cards/blob/master/preprocess.py
# # maybe not needed
# 
# def raw_text_cleaning(text):
#     ### Light
#     text = text.lower() # lowercase everything
#     text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
#     text = re.sub(r'https*\S+', ' ', text) # remove links
#     text = re.sub(r'http*\S+', ' ', text)
#     
#     # cleaning up text
#     text = re.sub(r'\'\w+', '', text) 
#     text = re.sub(r'\w*\d+\w*', '', text)
#     text = re.sub(r'\s{2,}', ' ', text)
#     text = re.sub(r'\s[^\w\s]\s', '', text)
# 
#     return text

In [2]:
## big loop for pdfs

# import relevant libraries

import tika
from tika import parser # pip install tika

import numpy as np
import pandas as pd

from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download('brown')


import re
import os
import glob

from bs4 import BeautifulSoup
import requests

# setup tika server instance
tika.initVM()

# define directory and extract file names
dir_path = "pdf/"
filenames = os.listdir(dir_path)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
## https://github.com/chrismattmann/tika-python/issues/191
#from io import StringIO
#from bs4 import BeautifulSoup
#from tika import parser



#file_data = []
#_buffer = StringIO()
#data = parser.from_file(dir_path+filenames[name], xmlContent= True)
#xhtml_data = BeautifulSoup(data['content'])




In [4]:
# setup database for ExxonMobil pdf files
# MAYBE try and optimize the code to not append to a .pd dataframe but rather a list object instead
column_names = {'original_text':    ['original text'],
                'cleaned_text':     ['cleaned text'],
                'climate_related':  ['0/1'],
                'downplaying':      ['0/1'],
                'source':           ['filename'],
                'type':             ['sentence/paragraph'],
                'year':             [2000]}

database_pdfs_exxon = pd.DataFrame(data = column_names)


database_pdfs_exxon_paragraphs = pd.DataFrame(data = column_names)
database_pdfs_exxon_paragraphs.drop(['original_text'], axis = 1, inplace = True)

#paragraph_index = 0 # for paragraph selection internal loop


for name in range(len(filenames)):

    print(filenames[name])

    # read the pdf from the corresponding file path
    filename = filenames[name]
    current_parsed = parser.from_file(dir_path+filenames[name], xmlContent= False)
    current_raw_text = current_parsed['content']

    blob_original_text = TextBlob(current_raw_text)

    # cleaning the sentences
    cleaned_full_text = re.sub(r'[.]\n\n', '. 777 ',current_raw_text) #looks for '.' followed by 'new line' to detect new paragraphs
    cleaned_full_text = re.sub(r'[7][7]\s\s','88',cleaned_full_text)
    cleaned_full_text = re.sub(r'\n\n',' ',cleaned_full_text)
    cleaned_full_text = re.sub(r'-\s','',cleaned_full_text)
    cleaned_full_text = re.sub(r'-\n','',cleaned_full_text)
    cleaned_full_text = re.sub(r'\n', ' ',cleaned_full_text)
    cleaned_full_text = re.sub(r'\s{2,}','',cleaned_full_text) # removes the whitespace infront of the starting sentence
    cleaned_full_text = re.sub(r'[•]\s', '', cleaned_full_text) # removes bulletpoints

    blob_cleaned_full_text = TextBlob(cleaned_full_text)
    
    current_paragraph = ""
    

    for i in range(len(blob_cleaned_full_text.sentences)):
        
        current_sentence = str(blob_cleaned_full_text.sentences[i])

        if re.search(r'\d{4}',filenames[name]):
            current_year = int(re.findall(r'\d{4}',filenames[name])[0])
        else:
            current_year = ''
        
        # check for missmatch with the original sentence data frame
        if i+1 < len(blob_cleaned_full_text.sentences):
            original_sentence = str(blob_original_text.sentences[i])
        else:
            original_sentence = "NA"
        

        # detecting paragraphs through special symbol
        if re.search(r'777\s',current_sentence):
            current_type = 'start of paragraph'
            
            #paragraph_index = paragraph_index + 1
            new_paragraph_row = [current_paragraph, 1, 1, filename, 'paragraph',current_year]
            database_pdfs_exxon_paragraphs.loc[len(database_pdfs_exxon_paragraphs)] = new_paragraph_row



            current_sentence = re.sub(r'^777\s','',current_sentence)
            current_paragraph = current_sentence
            

        elif re.search(r'788\s[A-Z]',current_sentence):
            current_type ='end of page (special case)'
            
            new_paragraph_row = [current_paragraph, 1, 1, filename, 'paragraph',current_year]
            database_pdfs_exxon_paragraphs.loc[len(database_pdfs_exxon_paragraphs)] = new_paragraph_row
            

            current_sentence = re.sub(r'^788\s','',current_sentence)
            current_paragraph = current_sentence


        elif re.search('788',current_sentence):
            current_type = 'end of page'
            
            new_paragraph_row = [current_paragraph, 1, 1, filename, 'paragraph',current_year]
            database_pdfs_exxon_paragraphs.loc[len(database_pdfs_exxon_paragraphs)] = new_paragraph_row
            
            current_sentence = re.sub(r'^788','NaN',current_sentence)
            current_paragraph = ''

        
        else:
            current_type = 'sentence'
            current_paragraph = current_paragraph + ' ' + current_sentence


        new_row = [original_sentence,current_sentence,1,1,filename,current_type,current_year]
        database_pdfs_exxon.loc[len(database_pdfs_exxon)] = new_row


database_pdfs_exxon['year'].replace({'':np.nan}, inplace = True)
database_pdfs_exxon_paragraphs['year'].replace({'':np.nan}, inplace = True)

1972-1-24_Mobil-nyt-66bn-mistake.pdf


2023-05-08 20:48:55,012 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to C:\Users\lucas\AppData\Local\Temp\tika-server.jar.
2023-05-08 20:50:21,282 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to C:\Users\lucas\AppData\Local\Temp\tika-server.jar.md5.
2023-05-08 20:50:22,360 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-05-08 20:50:27,366 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


1981-2-12-Mobil-nyt-endcoalnightmare.pdf
1981-7-30-Mobil-nyt-healthairstds.pdf
1981-8-13-Mobil-nyt-costofcleanair.pdf
1981-8-6-Mobil-nyt-canyonising.pdf
1982-3-26-Mobil-nyt-caa.pdf
1984-8-1-Mobil-nyt-Lies-They-Tell-Our-Children.pdf
1986-10-21-Mobil-lat-taxesmisstarget.pdf
1987-5-7-Mobil-nyt-rushtojudge.pdf
1989-7-6-Mobil-nyt-peoplewholivegreenhouses.pdf
1990-12-13-Mobil-nyt-cowsbullscleanair.pdf
1990-9-27-Mobil-nyt-deliveringaspromised.pdf
1992-4-2-Mobil-nyt-sensefmnonsense.pdf
1993-12-9-Mobil-nyt-envtorpolitics.pdf
1993-2-18-Mobil-nyt-whenthegoaliscleanair.pdf
1993-2-25-Mobil-nyt-apocalypse-no.pdf
1994-10-27-Mobil-nyt-whoindriversseat.pdf
1995-9-28-Mobil-nyt-skynotfalling.pdf
1996-12-12-Mobil-nyt-policyfortomorrow.pdf
1996-12-5-Mobil-nyt-electriccars.pdf
1996-7-18-Mobil-nyt-lessheatmorelight.pdf
1996-8-1-Mobil-nyt-allintogether.pdf
1997-10-16-Mobil-nyt-cnnslam.pdf
1997-10-23-Mobil-nyt-globalclimatechange.pdf
1997-10-30-Mobil-nyt-resetalarm.pdf
1997-11-13-Mobil-nyt-climateprudentapproa

### same loop but for IPCC pdf

In [3]:
# different folder selector
dir_path_ipcc = "ipcc/"
filenames_ipcc = os.listdir(dir_path_ipcc)


In [4]:
column_names = {'original_text':    ['original text'],
                'cleaned_text':     ['cleaned text'],
                'climate-related':  ['0/1'],
                'downplaying':      ['0/1'],
                'source':           ['filename'],
                'type':             ['sentence/paragraph'],
                'year':             ['2000']}

database_pdfs_ipcc = pd.DataFrame(data = column_names)


database_pdfs_ipcc_paragraphs = pd.DataFrame(data = column_names)
database_pdfs_ipcc_paragraphs.drop(['original_text'], axis = 1, inplace = True)

#paragraph_index = 0 # for paragraph selection internal loop


for name in range(len(filenames_ipcc)):

    print(filenames_ipcc[name])

    # read the pdf from the corresponding file path
    filename = filenames_ipcc[name]
    current_parsed = parser.from_file(dir_path_ipcc+filenames_ipcc[name])
    current_raw_text = current_parsed['content']

    blob_original_text = TextBlob(current_raw_text)

    # cleaning the sentences
    # cleaned_full_text = re.sub(r'[.]\n\n', '. 777 ',current_raw_text) #looks for '.' followed by 'new line' to detect new paragraphs
    # cleaned_full_text = re.sub(r'[7][7]\s\s','88',current_raw_text)
    # cleaned_full_text = re.sub(r'(\n\n)',' 777 ',current_raw_text) #maybe need to re-add the white space
    cleaned_full_text = re.sub(r'(\n\n)',' ',current_raw_text)
    cleaned_full_text = re.sub(r'-\n','',cleaned_full_text)
    cleaned_full_text = re.sub(r'\n', ' ',cleaned_full_text)
    cleaned_full_text = re.sub(r'\s{2,}',' ',cleaned_full_text) # removes the whitespace infront of the starting sentence
    cleaned_full_text = re.sub(r'-\s','5',cleaned_full_text)
    cleaned_full_text = re.sub(r'[•]\s', '', cleaned_full_text) # removes bulletpoints




    # special cleaning for IPCC "Language", e.g. {references in brackets}
    cleaned_full_text = re.sub(r'SPM.','',cleaned_full_text) #solves problem that TextBlob detects this "." as end of phrase
    cleaned_full_text = re.sub(r'\s\((.*?)\)','',cleaned_full_text)
    cleaned_full_text = re.sub(r'\[(.*?)\]\s', '', cleaned_full_text)
    cleaned_full_text = re.sub(r'CO,\s','CO2 ', cleaned_full_text)
    cleaned_full_text = re.sub(r'\{(.){3,}\},', '', cleaned_full_text)
    cleaned_full_text = re.sub(r'\{(.*?)\}','', cleaned_full_text)
    cleaned_full_text = re.sub(r'[�]', '', cleaned_full_text)
    cleaned_full_text = re.sub(r'\xa0', ' ', cleaned_full_text)



    blob_cleaned_full_text = TextBlob(cleaned_full_text)
    
    current_paragraph = ""
    

    for i in range(len(blob_cleaned_full_text.sentences)):
        
        current_sentence = str(blob_cleaned_full_text.sentences[i])

        if re.search(r'\d{4}',filenames_ipcc[name]):
            current_year = int(re.findall(r'\d{4}',filenames_ipcc[name])[0])
        else:
            current_year = ''


        # check for missmatch with the original sentence data frame
        if i+1 < len(blob_cleaned_full_text.sentences):
            original_sentence = str(blob_original_text.sentences[i])
        else:
            original_sentence = "NA"
        

       # # detecting paragraphs through special symbol
       # if re.search(r'^777\s',current_sentence):
       #     current_type = 'start of paragraph'
       #     
       #     #paragraph_index = paragraph_index + 1
       #     new_paragraph_row = [current_paragraph, 1, 0, filename, 'paragraph',current_year]
       #     database_pdfs_ipcc_paragraphs.loc[len(database_pdfs_ipcc_paragraphs)] = new_paragraph_row



       #     current_sentence = re.sub(r'^777\s','',current_sentence)
       #     current_paragraph = current_sentence
       #     

       # elif re.search(r'788\s[A-Z]',current_sentence):
       #     current_type ='end of page (special case)'
       #     
       #     new_paragraph_row = [current_paragraph, 1, 0, filename, 'paragraph',current_year]
       #     database_pdfs_ipcc_paragraphs.loc[len(database_pdfs_ipcc_paragraphs)] = new_paragraph_row
       #     

       #     current_sentence = re.sub(r'^788\s','',current_sentence)
       #     current_paragraph = current_sentence


       # elif re.search('788',current_sentence):
       #     current_type = 'end of page'
       #     
       #     new_paragraph_row = [current_paragraph, 1, 0, filename, 'paragraph',current_year]
       #     database_pdfs_ipcc_paragraphs.loc[len(database_pdfs_ipcc_paragraphs)] = new_paragraph_row
       #     
       #     current_sentence = re.sub(r'^788','-',current_sentence)
       #     current_paragraph = ''

       # 
       # else:
       #     current_type = 'sentence'
       #     current_paragraph = current_paragraph + ' ' + current_sentence




        new_row = [original_sentence,current_sentence,1,0,filename,current_type,current_year]
        database_pdfs_ipcc.loc[len(database_pdfs_ipcc)] = new_row



1995_IPCC_SR_AR2_Summary_Policymakers.pdf
2001_IPCC_SR_AR3_Total_Summary_Policymakers.pdf
2001_IPCC_SR_AR3_wg1_Summary_Policymakers.pdf
2001_IPCC_SR_AR3_wg2_Summary_Policymakers.pdf
2001_IPCC_SR_AR3_wg3_Summary_Policymakers.pdf
2007_IPCC_SR_AR4_Summary_Policymakers.pdf
2014_IPCC_SR_AR5_Headlines.pdf
2014_IPCC_SR_AR5_Summary_Policymakers.pdf
2022_IPCC_SR_AR6_WGIII_Headlines.pdf
2022_IPCC_SR_AR6_WGIII_Summary_Policymakers.pdf
2022_IPCC_SR_AR6_WGII_Headlines.pdf
2022_IPCC_SR_AR6_WGII_Summary_Policymakers.pdf


## scraping of Greenpeace data  

websites to scrape:
- https://www.greenpeace.org/usa/issues/a-world-beyond-fossil-fuels/
- https://www.greenpeace.org/usa/research/8-reasons-why-we-need-to-phase-out-the-fossil-fuel-industry/
- https://www.greenpeace.org/usa/issues/climate-change-impacts/
- https://www.greenpeace.org/usa/issues/california-climate-emergency/
- https://www.greenpeace.org/usa/issues/climate-leadership/
- https://www.greenpeace.org/usa/issues/renewable-energy/
- https://www.greenpeace.org/usa/campaigns/climate/
- https://www.greenpeace.org/usa/the-green-new-deal/
- https://www.greenpeace.org/usa/fighting-climate-chaos/issues/
- https://www.greenpeace.org/usa/fighting-climate-chaos/issues/coal/
- https://www.greenpeace.org/usa/fighting-climate-chaos/issues/oil/
- https://www.greenpeace.org/usa/fighting-climate-chaos/issues/fracking/
- https://www.greenpeace.org/usa/fighting-climate-chaos/issues/natural-gas/
- https://www.greenpeace.org/usa/fighting-climate-chaos/
- https://www.greenpeace.org/usa/fighting-climate-chaos/climate-science/
- https://www.greenpeace.org/usa/fighting-climate-chaos/renewable-energy-future/

In [13]:
greenpeace_websites = [ "https://www.greenpeace.org/usa/issues/a-world-beyond-fossil-fuels/",
                        "https://www.greenpeace.org/usa/research/8-reasons-why-we-need-to-phase-out-the-fossil-fuel-industry/",
                        "https://www.greenpeace.org/usa/issues/climate-change-impacts/",
                        "https://www.greenpeace.org/usa/issues/california-climate-emergency/",
                        "https://www.greenpeace.org/usa/issues/climate-leadership/",
                        "https://www.greenpeace.org/usa/issues/renewable-energy/",
                        "https://www.greenpeace.org/usa/campaigns/climate/",
                        "https://www.greenpeace.org/usa/the-green-new-deal/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/issues/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/issues/coal/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/issues/oil/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/issues/fracking/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/issues/natural-gas/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/climate-science/",
                        "https://www.greenpeace.org/usa/fighting-climate-chaos/renewable-energy-future/",
                        "https://www.greenpeace.org/usa/big-oil-forces-a-hold-on-california-law-ending-neighborhood-drilling/",
                        "https://www.greenpeace.org/usa/bidens-alarming-choice-to-allow-the-largest-sale-of-public-waters-for-offshore-oil-drilling-in-us-history/",
                        "https://www.greenpeace.org/usa/top-5-truly-scary-facts-about-our-climate-crisis/",
                        "https://www.greenpeace.org/usa/no-climate-no-deal-how-progressives-are-holding-the-line-for-the-full-build-back-better-act/",
                        "https://www.greenpeace.org/usa/global-heatwaves-are-fossil-fuel-driven-climate-chaos/",
                        "https://www.greenpeace.org/usa/why-net-zero-and-offsets-wont-solve-the-climate-crisis/",
                        "https://www.greenpeace.org/usa/joe-biden-marks-the-beginning-of-the-end-for-the-oil-industry/",
                        "https://www.greenpeace.org/usa/4-ways-to-shrink-the-fossil-fuel-industry-and-protect-the-climate/",
                        "https://www.greenpeace.org/usa/8-reasons-why-we-need-to-phase-out-the-fossil-fuel-industry/",
                        "https://www.greenpeace.org/usa/if-we-dont-stop-producing-fossil-fuels-we-wont-make-it/",
                        "https://www.greenpeace.org/usa/lets-talk-about-climate-change/",
                        "https://www.greenpeace.org/usa/heres-everything-wrong-with-scott-pruitts-plan-to-debate-climate-science/",
                        "https://www.greenpeace.org/usa/trumps-attack-on-the-paris-climate-agreement-has-officially-begun/",
                        "https://www.greenpeace.org/usa/4-actually-true-facts-about-jobs-renewable-energy-and-fossil-fuels/",
                        "https://www.greenpeace.org/usa/trumps-latest-executive-order-is-an-all-out-attack-on-clean-energy/",
                        "https://www.greenpeace.org/usa/trumps-refusal-to-act-on-climate-could-cost-trillions-of-dollars-every-year-for-decades/"
                        ]

In [14]:
column_names = {'original_text':    ['original text'],
                'cleaned_text':     ['cleaned text'],
                'climate-related':  ['0/1'],
                'downplaying':      ['0/1'],
                'source':           ['filename'],
                'type':             ['sentence/paragraph'],
                'year':             ['2000']}

database_pdfs_greenpeace = pd.DataFrame(data = column_names)



#paragraph_index = 0 # for paragraph selection internal loop


for website in range(len(greenpeace_websites)):

    print(greenpeace_websites[website])


    website_name = greenpeace_websites[website]

    # read the html from the corresponding url
    url = greenpeace_websites[website]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    filtered_for_class = soup.find_all("div", class_=["articleBody","primary-content", "post-content"])

    current_raw_text = filtered_for_class[0].text


    blob_original_text = TextBlob(current_raw_text)

    # cleaning the sentences
    cleaned_full_text = re.sub(r'(\xa0\n)',' ',current_raw_text)
    cleaned_full_text = re.sub(r'(\n\n)','',cleaned_full_text)
    cleaned_full_text = re.sub(r'\n', ' ',cleaned_full_text)
    cleaned_full_text = re.sub(r'\xa0', ' ',cleaned_full_text)

    

    # special cleaning for greenpeace "Language", e.g. {references in brackets}
   


    blob_cleaned_full_text = TextBlob(cleaned_full_text)
    
    current_paragraph = ""
    
    
    for i in range(len(blob_cleaned_full_text.sentences)):
        
        current_sentence = str(blob_cleaned_full_text.sentences[i])
        original_sentence = str(blob_original_text.sentences[i])

        # if re.search(r'\d{4}',filenames_ipcc[name]):
        #     current_year = int(re.findall(r'\d{4}',filenames_ipcc[name])[0])
        # else:
        #     current_year = ''


        new_row = [original_sentence,current_sentence,1,0,website_name,"sentence","XXXX"]
        database_pdfs_greenpeace.loc[len(database_pdfs_greenpeace)] = new_row


https://www.greenpeace.org/usa/issues/a-world-beyond-fossil-fuels/
https://www.greenpeace.org/usa/research/8-reasons-why-we-need-to-phase-out-the-fossil-fuel-industry/
https://www.greenpeace.org/usa/issues/climate-change-impacts/
https://www.greenpeace.org/usa/issues/california-climate-emergency/
https://www.greenpeace.org/usa/issues/climate-leadership/
https://www.greenpeace.org/usa/issues/renewable-energy/
https://www.greenpeace.org/usa/campaigns/climate/
https://www.greenpeace.org/usa/the-green-new-deal/
https://www.greenpeace.org/usa/fighting-climate-chaos/issues/
https://www.greenpeace.org/usa/fighting-climate-chaos/issues/coal/
https://www.greenpeace.org/usa/fighting-climate-chaos/issues/oil/
https://www.greenpeace.org/usa/fighting-climate-chaos/issues/fracking/
https://www.greenpeace.org/usa/fighting-climate-chaos/issues/natural-gas/
https://www.greenpeace.org/usa/fighting-climate-chaos/
https://www.greenpeace.org/usa/fighting-climate-chaos/climate-science/
https://www.greenpeac

In [17]:
# filter greenpeace sentences that are too short
database_pdfs_greenpeace['word_count'] = [len(sentence.split()) for sentence in database_pdfs_greenpeace['cleaned_text']]

In [23]:
database_pdfs_greenpeace_filtered = database_pdfs_greenpeace[database_pdfs_greenpeace['word_count'] >= 6]
database_pdfs_greenpeace_filtered.reset_index(drop = True, inplace= True)

In [25]:
# save the dataframes to .csv files
database_pdfs_ipcc.to_csv('ipcc_out.csv')
database_pdfs_exxon.to_csv('exxon_out.csv')
database_pdfs_exxon_paragraphs.to_csv('exxon_paragraphs_out.csv')
database_pdfs_greenpeace_filtered.to_csv('greenpeace_out.csv')

In [22]:
# save the dataframes to .parquet files
#database_pdfs_ipcc.to_parquet('ipcc_out.parquet')
database_pdfs_exxon[['cleaned_text','source','type','year']].to_parquet('exxon_out.parquet')
database_pdfs_exxon_paragraphs[['cleaned_text','source','type','year']].to_parquet('exxon_paragraphs_out.parquet')
#database_pdfs_greenpeace_filtered.to_parquet('greenpeace_out.parquet')